In [1]:
import pandas as pd
import numpy as np
import gzip


class SOMToolBox_Parse:
    
    def __init__(self, filename):
        self.filename = filename
    
    def read_weight_file(self,):
        df = pd.DataFrame()
        if self.filename[-3:len(self.filename)] == '.gz':
            with gzip.open(self.filename, 'rb') as file:
                df, vec_dim, xdim, ydim = self._read_vector_file_to_df(df, file)
        else:
            with open(self.filename, 'rb') as file:
                df, vec_dim, xdim, ydim = self._read_vector_file_to_df(df, file)

        file.close()            
        return df.astype('float64'), vec_dim, xdim, ydim


    def _read_vector_file_to_df(self, df, file):
        xdim, ydim, vec_dim, position = 0, 0, 0, 0
        for byte in file:
            line = byte.decode('UTF-8')
            if line.startswith('$'):
                xdim, ydim, vec_dim = self._parse_vector_file_metadata(line, xdim, ydim, vec_dim)
                if xdim > 0 and ydim > 0 and len(df.columns) == 0:
                    df = pd.DataFrame(index=range(0, ydim * xdim), columns=range(0, vec_dim))
            else:
                if len(df.columns) == 0 or vec_dim == 0:
                    raise ValueError('Weight file has no correct Dimensional information.')
                position = self._parse_weight_file_data(line, position, vec_dim, df)
        return df, vec_dim, xdim, ydim


    def _parse_weight_file_data(self, line, position, vec_dim, df):
        splitted=line.split(' ')
        try:
            df.values[position] = list(np.array(splitted[0:vec_dim]).astype(float))
            position += 1
        except: raise ValueError('The input-vector file does not match its unit-dimension.') 
        return  position


    def _parse_vector_file_metadata(self, line, xdim, ydim, vec_dim):
        splitted = line.split(' ')
        if splitted[0] == '$XDIM':      xdim = int(splitted[1])
        elif splitted[0] == '$YDIM':    ydim = int(splitted[1])
        elif splitted[0] == '$VEC_DIM': vec_dim = int(splitted[1])
        return xdim, ydim, vec_dim 
        

In [165]:
import itertools

def gen_neighbors(pt, dist_threshold = 2):
    neighbors = []
    nearest = np.round(pt)
    neighbors.append((nearest, np.linalg.norm(nearest-pt)))
    for offset in itertools.product(range(-dist_threshold, dist_threshold+1), repeat=2):
        if offset == (0,0):
            continue
        new_pt = pt + np.array(offset)
        neighbors.append((new_pt, np.linalg.norm(new_pt-pt)))
    neighbors.sort(key=lambda a: a[1])
    return neighbors

def direction(pt1, pt2):
    if (np.all(pt1 == pt2)): 
        return np.array([0,0])
    return (pt2 - pt1) / np.linalg.norm(pt2 - pt1)

def angle(v1, v2):
    a = np.arccos(np.dot(v1, v2))
    if np.isnan(a):
        print(f"{v1} {v2}")
    return a


def gen_feasible_neighbors(snapped, pt, dist_threshold = 2, corner_penalty = 2):
    neighbors = []
    nearest = np.round(pt)
    prev = snapped[-1]
    
    prev_direction = None
    if len(snapped) > 1:
        prev_direction = direction(snapped[-2], snapped[-1])
    
    if nearest[0] == prev[0] or nearest[1] == prev[1] or len(np.unique(nearest-prev)) == 1:
        penalty = 0
        new_dir = direction(prev, nearest)
        if prev_direction is not None and np.any(prev_direction != new_dir):
            penalty = angle(prev_direction, new_dir) * corner_penalty
        
        neighbors.append((nearest, np.linalg.norm(nearest-pt) + penalty))
    
    for offset in itertools.product(range(-dist_threshold, dist_threshold+1), repeat=2):
        if offset == (0,0):
            continue
        new_pt = nearest + np.array(offset)
                         
        if new_pt[0] == prev[0] or new_pt[1] == prev[1] or len(np.unique(new_pt-prev)) == 1:
            penalty = 0
            new_dir = direction(prev, new_pt)
            if prev_direction is not None and np.any(prev_direction != new_dir):
                penalty = angle(prev_direction, new_dir) * corner_penalty
            neighbors.append((new_pt, np.linalg.norm(new_pt-pt) + penalty))
        #else:
        #    print(f"REJ {prev} {new_pt} ({nearest})")

    #if len(neighbors) == 0:
    #    print(f"WARN: increasing threshold to {dist_threshold+1}")
    #    print(f"{prev} {pt} ({nearest})")
    #    neighbors = gen_feasible_neighbors(prev, pt, dist_threshold+1)
    neighbors.sort(key=lambda a: a[1])
    #print(f"{prev} {pt} {neighbors}")
    return neighbors

def snap_line(line, snapped, dist, lb):
    if len(line) == 0:
        return dist, snapped
    pt = line[0]
    best_line = None
    best = None
    #print(line)
    #print(snapped)
    neighbors = gen_feasible_neighbors(snapped, pt)
    for n_pt, n_dist in neighbors:
        if (dist + n_dist) > lb:
            continue
        n_snapped = snapped[:]
        n_snapped.append(n_pt)
        total_dist, new_snapped = snap_line(line[1:], n_snapped, dist + n_dist, lb)
        if total_dist is not None and total_dist < lb:
            lb = total_dist
            best = total_dist
            best_line = new_snapped
    return best, best_line
    

def snap(lines):
    dist_threshold = 3
    snapped_lines = []
    for idx, line in enumerate(lines):
        print(f"snapping line {idx+1}/{len(lines)}")
        start = line[0]
        neighbors = gen_neighbors(start)
        best_dist = 999999
        snapped = None
        for pt, dist in neighbors:
            n_dist, n_snapped = snap_line(line[1:], [pt], dist, best_dist)
            if n_dist is None:
                continue
            if n_dist < best_dist:
                #print(n_snapped)
                best_dist = n_dist
                snapped = n_snapped
        snapped_lines.append(snapped)
    return snapped_lines



In [166]:
import numpy as np
from scipy.spatial import distance_matrix, distance
from ipywidgets import Layout, HBox, Box, widgets, interact
import plotly.graph_objects as go
import matplotlib.pyplot as plt

from IPython.core.display import display #, HTML
#display(HTML("<style>.container { width:100% !important; }</style>")) 

class SomViz:
    
    def __init__(self, weights=[], m=None, n=None):
        self.weights = weights
        self.m = m
        self.n = n

    def umatrix(self, som_map=None, color="Viridis", interp = "best", title=""):
        um =np.zeros((self.m *self.n, 1))
        neuron_locs = list()
        for i in range(self.m):
            for j in range(self.n):
                neuron_locs.append(np.array([i, j]))
        neuron_distmat = distance_matrix(neuron_locs,neuron_locs)

        for i in range(self.m * self.n):
            neighbor_idxs = neuron_distmat[i] <= 1
            neighbor_weights = self.weights[neighbor_idxs]
            um[i] = distance_matrix(np.expand_dims(self.weights[i], 0), neighbor_weights).mean()

        if som_map is None:
            return self.plot(um.reshape(self.m,self.n), color=color, interp=interp, title=title)    
        else:
            som_map.data[0].z = um.reshape(self.m,self.n)

    def hithist(self, som_map=None, idata = [], color='RdBu', interp = "best", title=""):
        hist = [0] *self.n *self.m
        for v in idata: 
            position =np.argmin(np.sqrt(np.sum(np.power(self.weights - v, 2), axis=1)))
            hist[position] += 1    
        
        if som_map is None:
            return self.plot(np.array(hist).reshape(self.m,self.n), color=color, interp=interp, title=title)        
        else:
            som_map.data[0].z = np.array(hist).reshape(self.m,self.n)

    def component_plane(self, som_map=None, component=0, color="Viridis", interp = "best", title=""):
        if som_map is None:
            return self.plot(self.weights[:,component].reshape(-1,self.n), color=color, interp=interp, title=title)   
        else:
            som_map.data[0].z = self.weights[:,component].reshape(-1,n)
#
#    def sdh(self, som_map=None, idata=[], sdh_type=1, factor=1, draw=True, color="Cividis", interp = "best", title=""):
#        import heapq
#        sdh_m = [0] *self.m *self.n
#
#        cs=0
#        for i in range(0,factor): cs += factor-i
#
#        for vector in idata:
#            dist = np.sqrt(np.sum(np.power(self.weights - vector, 2), axis=1))
#            c = heapq.nsmallest(factor, range(len(dist)), key=dist.__getitem__)
#            if (sdh_type==1): 
#                for j in range(0,factor):  sdh_m[c[j]] += (factor-j)/cs # normalized
#            if (sdh_type==2):
#                for j in range(0,factor): sdh_m[c[j]] += 1.0/dist[c[j]] # based on distance
#            if (sdh_type==3): 
#                dmin = min(dist)
#                for j in range(0,factor): sdh_m[c[j]] += 1.0 - (dist[c[j]]-dmin)/(max(dist)-dmin)  
#
#        if som_map is None:
#            return self.plot(np.array(sdh_m).reshape(-1,self.n), color=color, interp=interp, title=title)      
#        else:
#            som_map.data[0].z = np.array(sdh_m).reshape(-1,self.n)
#        
#    def project_data(self,som_m=None, idata=[], title=""):
#        data_y = []
#        data_x = []
#        for v in idata:
#            position =np.argmin(np.sqrt(np.sum(np.power(self.weights - v, 2), axis=1)))
#            x,y = position % self.n, position // self.n
#            data_x.extend([x])
#            data_y.extend([y])
#            
#        if som_m is not None:
#            som_m.add_trace(go.Scatter(x=data_x, y=data_y, mode = "markers", marker_color='rgba(255, 255, 255, 0.8)',))
#    
#    def time_series(self, som_m=None, idata=[], wsize=50, title=""): #not tested
#             
#        data_y = []
#        data_x = [i for i in range(0,len(idata))]
#        
#        data_x2 = []
#        data_y2 = []
#        
#        qmin = np.Inf
#        qmax = 0
#        
#        step=1
#        
#        ps = []
#        for v in idata:
#            matrix = np.sqrt(np.sum(np.power(self.weights - v, 2), axis=1))
#            position = np.argmin(matrix)
#            qerror = matrix[position]
#            if qmin>qerror: qmin = qerror
#            if qmax<qerror: qmax = qerror
#            ps.append((position, qerror))
#       
#        markerc=[]    
#        for v in ps:
#            data_y.extend([v[0]])
#            rez = v[1]/qmax
# 
#            markerc.append('rgba(0, 0, 0, '+str(rez)+')') 
#            
#            x,y = v[0] % self.n, v[0] // self.n 
#            if    x==0: y = np.random.uniform(low=y, high=y+.1)
#            elif  x==self.m-1: y = np.random.uniform(low=y-.1, high=y)
#            elif  y==0: x = np.random.uniform(low=x, high=x+.1)
#            elif  y==self.n-1: x = np.random.uniform(low=x-.1, high=x)
#            else: x,y = np.random.uniform(low=x-.1, high=x+.1), np.random.uniform(low=y-.1, high=y+.1)                           
#            
#            data_x2.extend([x])
#            data_y2.extend([y]) 
#    
#        ts_plot = go.FigureWidget(go.Scatter(x=[], y=[], mode = "markers", marker_color=markerc, marker=dict(colorscale='Viridis', showscale=True, color=np.random.randn(500))))
#        ts_plot.update_xaxes(range=[0, wsize])
#
#        
#        ts_plot.data[0].x, ts_plot.data[0].y = data_x, data_y
#        som_m.add_trace(go.Scatter(x=data_x2, y=data_y2, mode = "markers",))
#  
#        som_m.layout.height = 500
#        ts_plot.layout.height = 500
#        som_m.layout.width = 500
#        ts_plot.layout.width = 1300
#        
#        return HBox([go.FigureWidget(som_m), go.FigureWidget(ts_plot)])
            

    def metro(self, som_map=None, bins=6):
        if som_map is None:
            som_map = self.umatrix(color='viridis', interp=None, title='U-matrix SOMToolBox')

        lines = []
        n_lines = self.weights.shape[1]
        for component in range(n_lines):
            raw = self.weights[:,component].reshape(-1,self.n)
            ranges = np.linspace(raw.min(), raw.max(), bins)
            binned = np.digitize(raw, ranges)
            stations = []
            for i in range(1, bins+1):
                match = np.argwhere(binned == i)
                stations.append(np.sum(match, axis=0)/match.shape[0])
            lines.append(stations)
        l1 = 1
        l2 = 3
        #print("#########BEFORE")
        #print(lines)
        lines2 = snap(lines)
        #print("#########AFTER")
        #print(lines)
        #xconv = np.convolve(list(zip(*lines[l1]))[1], list(zip(*lines[l2]))[1])
        #yconv = np.convolve(list(zip(*lines[l1]))[0], list(zip(*lines[l2]))[0])
        
        #conv = np.sum([xconv, yconv], axis=0)
        #conv = conv/np.max(conv)
        #plt.plot(np.arange(len(conv)), conv)
        #print(np.argwhere(conv >= 0.8).flatten())
        #som_map.add_trace(go.Scatter(x=[l[1] for l in line], y=[l[0] for l in line],
        #                             mode='lines+markers',
        #                             name=f'Component {i}')
        close_points = []
        distance_threshold = 1.0
        for idx1, line1 in enumerate(lines):
            for idx2, line2 in enumerate(lines):
                if idx1 == idx2:
                    continue
                close_points.append([])
                for i, pt1 in enumerate(line1):
                    for j, pt2 in enumerate(line2):
                        if np.linalg.norm(pt1-pt2) < distance_threshold:
                            close_points[-1].append((i,j))
                print(f'{idx1}<==>{idx2}: {close_points[-1]}')
                
        #for idx, line1 in enumerate(lines):
        #    close_points.append([])
        #    for i, pt1 in enumerate(line1):
        #        close_points[-1].append([])
        #        for line2 in lines[idx+1:]:
        #            for j, pt2 in enumerate(line2):
        #                if np.linalg.norm(pt1-pt2) < distance_threshold:
        #                    close_points[-1][-1].append((i,j))
        
        #for i, pairs in enumerate(close_points):
        #    for idx1, idx2 in pairs:
        #        lines[i
                                                         
        
        for i, line in enumerate(lines):
            #fig = self.plot(binned, interp=None, showscale=False)
            y, x = list(zip(*line))
            som_map.add_trace(go.Scatter(x=x, y=y, mode='lines+markers', name=f'Component {i}', line_shape='spline', line=dict(dash='dot')))
        
        for i, line in enumerate(lines2):
            #fig = self.plot(binned, interp=None, showscale=False)
            y, x = list(zip(*line))
            som_map.add_trace(go.Scatter(x=x, y=y, mode='lines+markers', name=f'Component {i}', line_shape='linear', line=dict(width=6), marker=dict(size=10, line=dict(width=2,color='black'))))


        return som_map
    
    def plot(self, matrix, color="Viridis",interp = "none", title="", showscale=False):
        return go.FigureWidget(go.Heatmap(z=matrix, zsmooth=interp, showscale=showscale, colorscale=color), layout=go.Layout(width=700, height=700,title=title, title_x=0.5,))


In [163]:
import minisom as som
from sklearn import datasets, preprocessing

#smap = SOMToolBox_Parse('iris.wgt.gz')
#smap, sdim, smap_x, smap_y = smap.read_weight_file()

# Visualizaton
#viz = SomViz(smap.values.reshape(-1,sdim), smap_y, smap_x)
m = 20
n = 20

iris = datasets.load_iris().data
min_max_scaler = preprocessing.MinMaxScaler()
iris = min_max_scaler.fit_transform(iris)

s = som.MiniSom(m, n, iris.shape[1], sigma=0.8, learning_rate=0.7)
s.train_random(iris, 10000, verbose=False)

In [167]:
# Visualizaton
viz = SomViz(s._weights.reshape(-1,4), m, n)
display(viz.metro(bins=10))

snapping line 1/4


<ipython-input-165-13429899f33f>:21: RuntimeWarning:

invalid value encountered in arccos



[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]


[-0.70710678 -0.70710678] [0.70710678 0.70710678]
[-0.70710678 -0.70710678] [0.70710678 0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[-0.70710678 -0.70710678] [0.70710678 0.70710678]
[-0.70710678 -0.70710678] [0.70710678 0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[-0.70710678 -0.70710678] [0.70710678 0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[-0.70710678 -0.70710678] [0.70710678 0.70710678]
[-0.70710678 -0.70710678] [0.70710678 0.70710678]
[-0.70710678 -0.70710678] [0.70710678 0.70710678]
[-0.70710678 -0.70710678] [0.70710678 0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[-0.70710678 -0.70710678] [0.70710678 0.70710678]
[-0.70710678 -0.70710678] [0.70710678 0.70710678]


[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[-0.70710678 -0.70710678] [0.70710678 0.70710678]
[-0.70710678 -0.70710678] [0.70710678 0.70710678]
[-0.70710678 -0.70710678] [0.70710678 0.70710678]
[-0.70710678 -0.70710678] [0.70710678 0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[-0.70710678 -0.70710678] [0.70710678 0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]
[0.70710678 0.70710678] [-0.70710678 -0.70710678]


FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

### Contributions to the Fitness function:
1. Geographical accuracy: How far a point on the metro map is from its original position. Influence can probably be calculated using a "rubber band" that tries to pull the stations towards their original positions.
2. Station merging: Stations that are close together should be merged into one crossover station. Can probably be modeled with an attraction force between stations that falls of with distance (e.g. 1/r^2)
3. Straightness of lines: The map should have as many straight elements as possible (reduce the number of kinks and corners)
4. Discretization of angles: Only a specified number of angles can be used for drawing the metro map.

### Notes
* Intermediate points could be introduced between the stops, so that we don't need a stop to create a corner. Those intermediate points could also be moved more freely as they are not bound to a geographical location like the stops are.

In [60]:
import pandas as pd
import minisom as som
from sklearn import datasets, preprocessing
#interp: False, 'best', 'fast', 
#color = 'viridis': https://plotly.com/python/builtin-colorscales/



#############################
######## miniSOM ############1/0
#############################
m=10
n=10

# Pre-processing 
iris = datasets.load_iris().data
min_max_scaler = preprocessing.MinMaxScaler()
iris = min_max_scaler.fit_transform(iris)

# Train
#s = som.MiniSom(m, n, iris.shape[1], sigma=0.8, learning_rate=0.7)
#s.train_random(iris, 10000, verbose=False)

# Visualizaton
#viz_miniSOM = SomViz(s._weights.reshape(-1,4), m, n)
#um1 = viz_miniSOM.umatrix(color='magma', interp='best', title='U-matrix miniSOM')

#display(um1)

##########################################
######## read from SOMToolBox ############
##########################################
trainedmap = SOMToolBox_Parse('iris.vec')
idata, idim, idata_x, idata_y = trainedmap.read_weight_file()

#display(idata)
#print(f'idim: {idim}')
#print(f'idata_x: {idata_x}')
#print(f'idata_y: {idata_y}')

smap = SOMToolBox_Parse('iris.wgt.gz')
smap, sdim, smap_x, smap_y = smap.read_weight_file()

#display(smap)
#print(f'sdim: {sdim}')
#print(f'smap_x: {smap_x}')
#print(f'smap_y: {smap_y}')

# Visualizaton
viz_SOMToolBox = SomViz(smap.values.reshape(-1,sdim), smap_y, smap_x)
#um2 = viz_SOMToolBox.umatrix(color='viridis', interp=None, title='U-matrix SOMToolBox') 
#um3 = viz_SOMToolBox.hithist(som_map=None, idata=idata, color='RdBu', interp="best", title="Hithist")

display(viz_SOMToolBox.metro())
#display(um2)
#display(um3)
#display(HBox([um2, um3]))

FileNotFoundError: [Errno 2] No such file or directory: 'iris.vec'